In [1]:
#!pip install ortools

import numpy as np
import datetime
import matplotlib.pyplot as plt

1. locate_call에서 np형태의 distance matrix 불러오기

In [2]:
time_matrix = np.load('../locate_call/강남역_경로매트릭스_24(분).npy')
d = time_matrix.tolist()
d

[[0,
  1924,
  2868,
  1770,
  2118,
  2274,
  2222,
  629,
  1307,
  3397,
  2056,
  2428,
  3430,
  2240,
  2394,
  2462,
  2433,
  2180,
  1802,
  2339,
  2907,
  945,
  2930,
  2174],
 [2642,
  0,
  1439,
  2382,
  1062,
  1472,
  1181,
  2823,
  3106,
  3064,
  1906,
  3888,
  3143,
  4620,
  3047,
  3424,
  4099,
  2478,
  3543,
  3418,
  1960,
  3153,
  4103,
  2434],
 [3059,
  1274,
  0,
  2798,
  1634,
  2105,
  924,
  3240,
  3523,
  3386,
  2541,
  4304,
  3466,
  4962,
  3370,
  4021,
  4422,
  2895,
  3960,
  3740,
  862,
  3570,
  3734,
  2757],
 [1456,
  2029,
  2972,
  0,
  2223,
  2393,
  2327,
  996,
  896,
  2442,
  2605,
  1701,
  2522,
  2344,
  1171,
  1214,
  2063,
  2446,
  3346,
  1301,
  3011,
  1906,
  1930,
  1620],
 [2720,
  838,
  1777,
  2459,
  0,
  2422,
  594,
  2900,
  3184,
  3364,
  1623,
  3965,
  3443,
  4939,
  3347,
  3502,
  4399,
  2196,
  3219,
  3718,
  1816,
  3230,
  4250,
  2734],
 [2364,
  1489,
  1862,
  2546,
  2174,
  0,
  1851,
  255

In [3]:
import pickle

with open("../locate_call/정류소명_리스트.pkl","rb") as f:
    name_lst = pickle.load(f)

In [4]:
"""Simple Pickup Delivery Problem (PDP) + Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = d
    data['demands'] = [0,2] + [1 for i in range(22)] 
    data['vehicle_capacities'] = [4,4,4,4,4,4]
    data['num_vehicles'] = 6
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_time = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_time = 0
        route_time_lst = [0]
        route_load = 0
        i=0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_time_lst.append(routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id))
            route_time += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            
            plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3})    ==>    '.format(name_lst[node_index], data['vehicle_capacities'][vehicle_id] - route_load,data['demands'][node_index],datetime.timedelta(seconds = route_time_lst[i]))
            i+=1
        plan_output += 'Duration of the route: {}분\n'.format((datetime.timedelta(seconds = route_time-route_time_lst[-1])))
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_time += route_time
        total_load += route_load
    print('Total Duration of all routes: {}m'.format(total_time))
    print('Total load of all routes: {}'.format(total_load))


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # 1-1. Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # 1-2.Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        30000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100) #이거 의미?

    # 2-1.Define cost of each node.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)

    # 2-2.Add Capacity constraint.
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    

    # Print solution on console.
    if solution:
      solution_dict =  print_solution(data, manager, routing, solution)
    else:
      print(solution)
  
if __name__ == '__main__':
    main()

Objective: 887756
Route for vehicle 0:
 정류장 이름 : 강남역  (탑승객 수: 4, 하차수: 0, 소요시간: 0:00:00)    ==>     정류장 이름 : 공항중학교.마곡엠밸리9,10단지  (탑승객 수: 3, 하차수: 1, 소요시간: 0:48:50)    ==>     정류장 이름 : 증미역  (탑승객 수: 2, 하차수: 1, 소요시간: 0:17:03)    ==>     정류장 이름 : 구로세관  (탑승객 수: 1, 하차수: 1, 소요시간: 0:24:51)    ==>     정류장 이름 : 도림사거리  (탑승객 수: 0, 하차수: 1, 소요시간: 0:14:31)    ==>    Duration of the route: 1:45:15분
Load of the route: 4

Route for vehicle 1:
 정류장 이름 : 강남역  (탑승객 수: 4, 하차수: 0, 소요시간: 0:00:00)    ==>     정류장 이름 : 신내동성3차아파트  (탑승객 수: 3, 하차수: 1, 소요시간: 0:35:18)    ==>     정류장 이름 : 서울과학기술대학교정문앞  (탑승객 수: 2, 하차수: 1, 소요시간: 0:09:54)    ==>     정류장 이름 : 명일동국민은행.래미안솔베뉴  (탑승객 수: 1, 하차수: 1, 소요시간: 0:29:11)    ==>     정류장 이름 : 일자산입구.둔촌도서관  (탑승객 수: 0, 하차수: 1, 소요시간: 0:10:01)    ==>    Duration of the route: 1:24:24분
Load of the route: 4

Route for vehicle 2:
 정류장 이름 : 강남역  (탑승객 수: 4, 하차수: 0, 소요시간: 0:00:00)    ==>     정류장 이름 : LH3.4단지  (탑승객 수: 3, 하차수: 1, 소요시간: 0:15:45)    ==>     정류장 이름 : 독산한신코아.한신아파트  (탑승객 수: 2, 하차수: 1, 소요시간: